<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-2-Building-Simple-Agentic-AI-Systems/M2LC3_Build_a_Text2SQL_Agentic_AI_System_with_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Text2SQL  Agentic AI System with LangGraph

### Building a Text2SQL Agentic AI System

In this project, we will design a **Text2SQL Agentic AI System** to enhance the interaction with databases using natural language queries. This workflow will leverage **LangGraph**, and build the agent from scratch. The workflow consists of the following components:

1. **Agent System Prompt**: The agent is designed to interpret user questions and generate syntactically correct SQL queries. Key rules include:
   - Limiting results to at most 10 unless specified by the user.
   - Querying only relevant columns instead of retrieving all columns from a table.
   - Ensuring plain text SQL query generation without additional formatting or code blocks.

2. **SQL Database Tools**: The system uses tools to interact with the database schema and validate queries:
   - **DB_QUERY**: Executes the generated SQL query.
   - **DB_SCHEMA**: Fetches the schema details of the database.
   - **DB_LIST_TABLES**: Lists all tables in the database.
   - **DB_QUERY_CHECKER**: Validates the correctness of the SQL query before execution.

3. **SQL Database Engine**: Executes the validated SQL queries and retrieves the required data from the database.

4. **ReAct Reasoning Framework**: The agent combines reasoning with actions to ensure accurate query formulation and execution.

5. **Final Response**: The results are processed and returned to the user in a human-friendly format.

By using LangGraph, this system ensures a more reliable, efficient, and user-friendly interaction with SQL databases, making it easier to extract meaningful insights from structured data.

![](https://i.imgur.com/4PNKql9.png)

## Install OpenAI, LangChain and LangGraph dependencies

In [ ]:
!pip install langgraph==0.6.4 langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 --quiet

## Install SQLite Server

In [ ]:
!apt-get install sqlite3 -y

## Setup Authentication and LLM Client

Here we authenticate and connect to necessary LLM Clients using OpenAI Authentication

In [ ]:
import os
import getpass

# OpenAI API Key (for chat & embeddings)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key (https://platform.openai.com/account/api-keys):\n")

## Get SQL DB

In [ ]:
# in case of issues download from https://drive.google.com/file/d/1_BJL-X6I5RTQjFc5WySKNSoFHkb1EFc8/view?usp=sharing and upload
!gdown 1_BJL-X6I5RTQjFc5WySKNSoFHkb1EFc8

## Create Operational Insights Database

In [ ]:
!sqlite3 --version

In [ ]:
!sqlite3 operations_insights.db "SELECT name FROM sqlite_master WHERE type='table';"

In [ ]:
%%bash
sqlite3 operations_insights.db <<EOF
.headers on
.mode column
SELECT * FROM Department LIMIT 10;
EOF

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///operations_insights.db")
db

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())

## 🧠 Overview of the Operations Insights Database

In this project, we will utilize an **Operations Insights Database** to demonstrate the capabilities of a Text2SQL workflow designed for operational diagnostics in enterprise workflows. The database schema includes the following entities and relationships:

1. **Department**: Contains details of business departments such as Finance Operations, Procurement, Claims Processing, etc.

2. **Workflow**: Represents business processes (e.g., Invoice Processing, Vendor Onboarding) and links them to departments.

3. **AuditLog**: Tracks historical audit entries including SLA issues, root causes, and recommended remediations, typically at a quarterly level.

4. **RoutingGap**: Captures live unresolved exceptions or routing failures in workflows, including severity, age, and status.

5. **FollowUpAction**: Logs corrective actions assigned to departments to resolve specific routing issues, with assignees and due dates.

### 🔄 Relationships:
- A **Department** can own multiple **Workflows**.
- A **Workflow** can have multiple **AuditLog** entries and **RoutingGaps**.
- A **RoutingGap** may result in one or more **FollowUpActions**.
- **FollowUpActions** are assigned by **Department** and linked to a specific **RoutingGap**.

This database schema is ideal for powering an agent that supports:
- Workflow audit analysis
- SLA breach diagnostics
- Routing exception monitoring
- Action tracking and reporting

The agent can convert natural language questions into SQL queries to retrieve relevant records and offer insights into enterprise process health.



![](https://i.imgur.com/O20Ds8c.png)

In [ ]:
db.run("SELECT * FROM Department LIMIT 10;", include_columns=True)

In [ ]:
db.run("SELECT * FROM Workflow LIMIT 10;", include_columns=True)

In [ ]:
db.run("SELECT * FROM AuditLog LIMIT 10;", include_columns=True)

In [ ]:
db.run("SELECT * FROM RoutingGap LIMIT 10;", include_columns=True)

In [ ]:
db.run("SELECT * FROM FollowUpAction LIMIT 10;", include_columns=True)

In [ ]:
print(db.get_table_info(table_names=['Department', 'Workflow']))

## Load up SQL Tools

The system uses tools to interact with the database schema and validate queries:
   - **DB_QUERY**: Executes the generated SQL query.
   - **DB_SCHEMA**: Fetches the schema details of the database.
   - **DB_LIST_TABLES**: Lists all tables in the database.
   - **DB_QUERY_CHECKER**: Validates the correctness of the SQL query before execution.

![](https://i.imgur.com/iVSrFE3.png)   

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sql_tools = sql_toolkit.get_tools()

sql_tools

In [ ]:
for tool in sql_tools:
    print(tool.name, '->', tool.description)

In [ ]:
llm_with_tools = llm.bind_tools(tools=sql_tools)

## Create ReAct Agent Instruction Prompt

In [ ]:
from langchain_core.messages import SystemMessage

AGENT_INSTRUCTIONS = r"""
You are an agent designed to interact with a SQLite database that tracks operational audits, workflow gaps,
and follow-up actions in an enterprise context.

You must generate syntactically correct SQL queries and then use them to answer user questions.

---

The database has the following important relationships:

- `Department(DepartmentID, Name)`
  → Maps department names (e.g., Finance Operations)

- `Workflow(WorkflowID, Name, DepartmentID)`
  → Links specific workflows to departments

- `AuditLog(AuditID, WorkflowID, Quarter, Issue, RootCause, Recommendation)`
  → Stores historical SLA failures, their causes and recommended remediations

- `RoutingGap(GapID, WorkflowID, Description, AgeHours, Severity, Status)`
  → Lists unresolved operational issues in workflows

- `FollowUpAction(ActionID, GapID, DepartmentID, AssignedTo, ActionDescription, DueDate, Status)`
  → Tracks who is assigned to fix what, and by when

---

Use JOINs to resolve cross-table queries. For example:
- To get all routing gaps for a department, join: `RoutingGap → Workflow → Department`
- To retrieve follow-up actions by department, join: `FollowUpAction → Department`
- To investigate audit reasons for delays, join: `AuditLog → Workflow → Department`

---

Important Instructions for a typical flow:
- Start by listing the available tables ( use `sql_db_list_tables` tool) to understand what tables are there
- Then inspect relevant table schemas using (`sql_db_schema` tool) based on your understanding of the query
- Only query relevant fields; never use `SELECT *`
- Always limit the result to 10 rows unless otherwise specified
- Always verify your SQL syntax using the `sql_db_query_checker` tool before execution
- Finally use the `sql_db_query` to execute your SQL query on the database and get relevant information
- Never generate INSERT, UPDATE, DELETE, or DROP statements

---

Output Format:
- Using above instructions create the final SQL query **as plain text**, no code blocks, no SQL markers or markdown and execute it
- Analyze results and create your final response based on the user query.
- Response should be easy to understand for the user, only use context from the results, do not add unnecessary suggestions out of context.
- Escape markdown-sensitive characters (e.g. $ → \$) when presenting the final response
- Do NOT include SQL queries in the final response

Example query:
SELECT Issue, RootCause FROM AuditLog A JOIN Workflow W ON A.WorkflowID = W.WorkflowID JOIN Department D ON W.DepartmentID = D.DepartmentID WHERE D.Name = 'Procurement & Vendor Mgmt' LIMIT 10
"""

SYS_PROMPT = SystemMessage(content=AGENT_INSTRUCTIONS)

## Create Text2SQL ReAct Agent

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import display, Image, Markdown

# Create the node function that handles reasoning and planning using the LLM
def tool_calling_llm(state: State) -> State:
    # Extract the current conversation history from the state
    current_state = state["messages"]

    # Prepend the system instructions to the current message history
    state_with_instructions = [SYS_PROMPT] + current_state

    # Call the LLM to generate a new message (either a response or a tool call request)
    response = [llm_with_tools.invoke(state_with_instructions)]

    # Return the updated state containing the new message
    return {"messages": response}

# Build the graph
builder = StateGraph(State)

# Add nodes
builder.add_node("agent", tool_calling_llm)
builder.add_node("tools", ToolNode(tools=sql_tools))

# Add edges
builder.add_edge(START, "agent")
# Conditional edge
builder.add_conditional_edges(
    "agent",
    tools_condition, # conditional routing function
    {
        "tools": "tools", # If the latest message (result) from LLM is a tool call request -> tools_condition routes to tools
        "__end__": END # If the latest message (result) from LLM is a not a tool call -> tools_condition routes to END
    }
)
builder.add_edge("tools", "agent") # this is the key feedback loop in the agentic system

# Compile Agent Graph
text2sql_agent = builder.compile()

In [ ]:
text2sql_agent

## Run and Test Text2SQL Agent

In [ ]:
# get agent streaming utils
!gdown 1dSyjcjlFoZpYEqv4P9Oi0-kU2gIoolMB

In [ ]:
from agent_utils import format_message

def call_text2sql_agent(query, verbose=False):

    for event in text2sql_agent.stream(
        {"messages": [HumanMessage(content=query)]},
        stream_mode='values' #returns full agent state with all messages including updates
    ):
        if verbose:
            format_message(event["messages"][-1])

    print('\n\nFinal Response:\n')
    display(Markdown(event["messages"][-1].content))
    return event["messages"]


In [ ]:
query = "Why did Finance miss SLA in Q1?"
response = call_text2sql_agent(query=query,
                               verbose=True)

In [ ]:
response = call_text2sql_agent(query="Which department has the highest unresolved issues",
                               verbose=False)

In [ ]:
response = call_text2sql_agent(query="What open issues exist in procurement?",
                               verbose=True)

In [ ]:
response = call_text2sql_agent(query="Which workflows had the most audits?",
                    verbose=False)

In [ ]:
response = call_text2sql_agent(query="Show me unresolved issues older than 48 hours.",
                               verbose=True)